In [25]:
# !pip install --user xmltodict
#!pip install elasticsearch==7.17

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 385 kB 10.6 MB/s eta 0:00:01
  Attempting uninstall: elasticsearch
    Found existing installation: elasticsearch 8.7.0
    Uninstalling elasticsearch-8.7.0:
      Successfully uninstalled elasticsearch-8.7.0


In [1]:
import random
import pytz
import shapely
from shapely.geometry import Point
import elasticsearch
from datetime import datetime
import time


In [4]:
# define mapping
mapping = {
        "mappings": {
            #"mydocs": {
                "properties": {
                    "Date": {
                     "type": "date"
                    },
                    "Destination": {
                     "type": "geo_point"
                    },
                    "Origin": {
                      "type": "geo_point"
                    },
                    "Title": {
                      "type": "keyword"
                    },
                }
            #}
       }
}

In [2]:
!curl kooplex-fiek.elte.hu:9100

{
  "name" : "adb999e53fb9",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "q0KmC5CxQ0K34Sg8v1ZNqw",
  "version" : {
    "number" : "7.8.0",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "757314695644ea9a1dc2fecd26d1a43856725e65",
    "build_date" : "2020-06-14T19:35:50.234439Z",
    "build_snapshot" : false,
    "lucene_version" : "8.5.1",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [3]:
# load Elasticsearch
try:
    es = elasticsearch.Elasticsearch(["http://kooplex-fiek.elte.hu:9100"])
    print(es.info())
except:
    print( "unable to es")

{'name': 'adb999e53fb9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'q0KmC5CxQ0K34Sg8v1ZNqw', 'version': {'number': '7.8.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '757314695644ea9a1dc2fecd26d1a43856725e65', 'build_date': '2020-06-14T19:35:50.234439Z', 'build_snapshot': False, 'lucene_version': '8.5.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [7]:
# create index mass_traffic
try:
    response = es.indices.create(index="random_point2",body=mapping, ignore=400)
except Exception as e:
    print(e)

In [8]:
response

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'random_point2'}

In [ ]:
now = pytz.utc.localize(datetime.utcnow())
id=1
latr1, latr2, lonr1, lonr2 = [random.uniform(-10,10) for i in range(4)]
while True:
    try:
        new_body = {}
        new_body["Date"] =  datetime.strftime(now,'%Y-%m-%dT%H:%M:%S.%fZ')
        p = Point((random.uniform(0.2+latr1,23+latr1), random.uniform(45.5+lonr1,49+lonr1))) 
        new_body["Destination"] = { "lat": p.x, 'lon': p.y}                
        p = Point((random.uniform(-43+latr2,-10+latr2), random.uniform(25.5+lonr2,39+lonr2))) 
        new_body["Origin"] = {"lat": p.x, 'lon': p.y}              

        new_body["Title"] = random.choice(['flight1, flight2, flight3'])
        id+=1


        print(new_body)
        res = es.index(index="random_point", doc_type="_doc", id=id, body=new_body)
        print( "------------------------------------------")
        print(res)
    except Exception as e:
        print( "------------------------------------------")
        print("unable to index in elasticsearch")
        print( "------------------------------------------")
        error = "{0}: {1}".format(id,e)
        print( error)
        print( "------------------------------------------")
    print( "all done, starting again soon")
    time.sleep(5)
    if id==30:
        id=1
        latr1, latr2, lonr1, lonr2 = random(4)
        latr1, latr2, lonr1, lonr2 = [random.uniform(-10,10) for i in range(4)]


## Queries DSL

GET random_point/_search
{
   "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_bounding_box": {
          "Origin": {
            "top_left": {
              "lat": -28.6603547,
              "lon": 0.2268691
            },
            "bottom_right": {
              "lat": -48.3658642,
              "lon": -26.9556643
            }
          }
        }
      }
    }
  }
}



In [4]:
es.ping()

True

In [5]:
# Search in random_point index
res = es.search(index="random_*")
res

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 30, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'random_point',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_source': {'Date': '2021-05-04T14:38:34.784436Z',
     'Destination': {'lat': 5.879576016187655, 'lon': 46.97323166969407},
     'Origin': {'lat': -36.69539171970964, 'lon': 30.745200421293113},
     'Title': 'flight1, flight2, flight3'}},
   {'_index': 'random_point',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.0,
    '_source': {'Date': '2021-05-04T14:38:34.784436Z',
     'Destination': {'lat': 4.363029482118389, 'lon': 47.33500886447054},
     'Origin': {'lat': -21.19022835197835, 'lon': 34.879807677623276},
     'Title': 'flight1, flight2, flight3'}},
   {'_index': 'random_point',
    '_type': '_doc',
    '_id': '4',
    '_score': 1.0,
    '_source': {'Date': '2021-05-04T14:38:34.784436Z',
     'Destinat

In [6]:
res=es.get(index='random_point',doc_type='_doc', id=2)
res

{'_index': 'random_point',
 '_type': '_doc',
 '_id': '2',
 '_version': 8,
 '_seq_no': 308,
 '_primary_term': 4,
 'found': True,
 '_source': {'Date': '2021-05-04T14:38:34.784436Z',
  'Destination': {'lat': 5.879576016187655, 'lon': 46.97323166969407},
  'Origin': {'lat': -36.69539171970964, 'lon': 30.745200421293113},
  'Title': 'flight1, flight2, flight3'}}